In [1]:
from ipywidgets import Label
import ephem

from datetime import datetime, timedelta

import time

import numpy as np

import odrive
from odrive.enums import *

import matplotlib.pyplot as plt

import adi

# Setup Dish Control

In [2]:
odrv0 = odrive.find_any()

In [3]:
print(str(odrv0.vbus_voltage))

24.176366806030273


In [4]:
# Find the index of the Encoder for precice start up positioning.
# This presumes the dish is parked just past the index pulse as the homed revolution
odrv0.axis0.requested_state = AXIS_STATE_ENCODER_INDEX_SEARCH
odrv0.axis1.requested_state = AXIS_STATE_ENCODER_INDEX_SEARCH

In [5]:
# Assumes we are sending commands at ~10Hz so set postion filter bandwidht to 1/2 that
odrv0.axis0.controller.config.input_filter_bandwidth = 5
odrv0.axis1.controller.config.input_filter_bandwidth = 5

In [6]:
odrv0.clear_errors()
odrv0.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
odrv0.axis0.controller.config.input_mode = INPUT_MODE_POS_FILTER

In [ ]:
# Axis 1 is not doing the derbin
odrv0.clear_errors()
odrv0.axis1.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE

In [14]:
odrv0.clear_errors()
odrv0.axis1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
odrv0.axis1.controller.config.input_mode = INPUT_MODE_POS_FILTER

## Test Movement
Run the cells below to check that each dish axis moves ~5 degrees and back

In [8]:
odrv0.axis0.controller.input_pos = 5

In [9]:
odrv0.axis0.encoder.pos_estimate

4.84821891784668

In [10]:
odrv0.axis0.controller.input_pos = 0

In [11]:
odrv0.axis0.encoder.pos_estimate

-0.08560752868652344

In [15]:
odrv0.axis1.controller.input_pos = 5

In [17]:
odrv0.axis1.encoder.pos_estimate

5.019256591796875

In [18]:
odrv0.axis1.controller.input_pos = 0

In [19]:
odrv0.axis1.encoder.pos_estimate

-0.4161682724952698

# Start Tracking

In [127]:
el_label = Label()
display(el_label)

Label(value='')

ERROR! Session/line number was not unique in database. History logging moved to new session 169

In [128]:
dish = ephem.Observer()
dish.lon = '-105.251825'
dish.lat = '40.010886'
dish.elevation = 1605 # Meters

sat = ephem.readtle('CIRBE',
    '1 56188U 23054L   23289.65931123  .00032081  00000-0  11665-2 0  9991',
    '2 56188  97.3883 183.2111 0007421 337.0021  23.0886 15.28169498 28690'
)

pass_start_elevation = 8

def get_current_time():
    return datetime.utcnow()

dish.date = get_current_time()
sat.compute(dish)

el_label.value = f'{np.degrees(sat.alt):3.2f} EL {np.degrees(sat.az):3.2f} AZ F'

# Wait for satellite to come above horizon
while sat.alt < np.radians(pass_start_elevation):

    time.sleep(1)
    dish.date = get_current_time()
    sat.compute(dish)

    el_label.value = f'{np.degrees(sat.alt):3.2f} EL {np.degrees(sat.az):3.2f} AZ W'

times = []
az_tgt = []
el_tgt = []
az_meas = []
el_meas = []

# Track satellite and record data
while sat.alt >= np.radians(pass_start_elevation):

    current_time = get_current_time()
    dish.date = current_time
    sat.compute(dish)

    sat_az = sat.az + np.radians(15.4)
    sat_el = sat.alt

    el_label.value = f'{np.degrees(sat_el):3.2f} EL {np.degrees(sat_az):3.2f} AZ R'

    # Handle crossing 360
    if len(az_tgt) == 0:
        az_last = np.degrees(sat_az)
    else:
        az_last = az_tgt[-1]
    az_next = np.degrees(sat_az)
    az_diff = az_next - az_last
    az_diff = (((az_diff + 180) % 360 ) - 180)

    az_tgt.append(az_last + az_diff)
    el_tgt.append(np.degrees(sat_el))

    # TODO offset correction
    
    a0_tgt = (az_tgt[-1] - 180) * (3199 / 3600)
    a1_tgt = (90 - el_tgt[-1]) * (3199 / 3600)

    odrv0.axis0.controller.input_pos = a0_tgt
    odrv0.axis1.controller.input_pos = a1_tgt

    a0_pos = odrv0.axis0.encoder.pos_estimate
    a1_pos = odrv0.axis1.encoder.pos_estimate

    az_meas.append( (a0_pos /  (3199 / 3600) ) + 180 )
    el_meas.append( 90 - (a1_pos / (3199 / 3600) ) )
    
    times.append(current_time)

# Park Dish

In [136]:
odrv0.axis0.controller.input_pos = 0
odrv0.axis1.controller.input_pos = 0